# Loading necessary Libraries

In [107]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style

# Loading Dataset

In [108]:
# Reading Data # First Column is the index
df = pd.read_csv('../00_Data/online_retailed_precleaned.csv', index_col=0)

In [109]:
# Shape of data
df.shape

(397656, 8)

In [110]:
# First five rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,3249,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,4048,United Kingdom
1,536365,2649,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,4048,United Kingdom
2,536365,2855,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,4048,United Kingdom
3,536365,2803,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,4048,United Kingdom
4,536365,2802,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,4048,United Kingdom


# Preparation: Binary Dataset

Main Steps

1. Remove all cancelled orders
2. Aggregate all User - Item Interactions & Create Flag Column for Purchases
3. Split into Train & Test Data

# 1. Removing Cancelled Orders

In [111]:
# Filter out rows with cancelled orders
df_non_cancelled = df[~df['InvoiceNo'].str.startswith('C')]

In [112]:
# Check Shape after Removal
df_non_cancelled.shape

(389326, 8)

# 2. Aggregate User - Item Interactions - Create Flag Column for Purchased Items

In [113]:
# Calculate Count of all non-cancelled interactions between Users & Items
df_binary = df_non_cancelled.groupby(['StockCode', 'CustomerID']).agg({'CustomerID': 'count'})

# Rename Columns 
df_binary.columns = ['purchased']

# Reset Index
df_binary.reset_index(inplace=True)

# Set all purchases to 1 
df_binary['purchased'] = 1 

# Check first five columns
df_binary.head()

,StockCode,CustomerID,purchased
0,0,86,1
1,0,132,1
2,0,191,1
3,0,238,1
4,0,270,1


In [106]:
# Sanity Check 
df_binary.purchased.describe()

count    259431.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: purchased, dtype: float64

In [114]:
# Print Unique Users & Items for Sanity Check  
print("Users: \t",df_binary['CustomerID'].nunique())
print("Items: \t",df_binary['StockCode'].nunique())

Users: 	 3697
Items: 	 2743


# 3. Split into Train & Test Data

In [124]:
# Load the Python Splitter
from recommenders.datasets.python_splitters import python_stratified_split

In [129]:
train, test = python_stratified_split(df_binary, ratio=0.8, filter_by='user', min_rating=10, col_user='CustomerID', col_item='StockCode', seed=1)
#train, test = train_test_split(df_binary, random_state=1, test_size=0.25)

In [130]:
# Sanity Check number of unique Users & Items in Train
print("Users Train: \t",train['CustomerID'].nunique())
print("Items Train: \t",train['StockCode'].nunique())
# Sanity Check number of unique Users & Items in Test
print("Users Test: \t",test['CustomerID'].nunique())
print("Items Test: \t",test['StockCode'].nunique())

Users Train: 	 3692
Items Train: 	 2743
Users Test: 	 3692
Items Test: 	 2716


In [131]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
train_items = set(train['StockCode'].unique())
test_items = set(test['StockCode'].unique())

# Find the intersection of the sets from step 1
common_items = train_items.intersection(test_items)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
train = train[train['StockCode'].isin(common_items)]
test = test[test['StockCode'].isin(common_items)]

In [132]:
# Sanity Check number of unique Users & Items in Train
print("Users Train: \t",train['CustomerID'].nunique())
print("Items Train: \t",train['StockCode'].nunique())
# Sanity Check number of unique Users & Items in Test
print("Users Test: \t",test['CustomerID'].nunique())
print("Items Test: \t",test['StockCode'].nunique())

Users Train: 	 3692
Items Train: 	 2716
Users Test: 	 3692
Items Test: 	 2716


In [133]:
# Shapes to Check 
print("Shape of Train:", train.shape)
print("Shape of Test: \t", test.shape)

Shape of Train: (207084, 3)
Shape of Test: 	 (51892, 3)


In [122]:
# Save as csv 
train.to_csv('../00_Data/online_retail_train.csv')
test.to_csv('../00_Data/online_retail_test.csv')


# Preparation: Pseudo-Rating Dataset

**Main Steps**

1. Create a flag column for purchased items
2. Decrease value of cancelled purchases 
3. Filter out cancelled combinations from initial Dataset
4. Increase value of repurchased items
5. Insert negative value for non-purchased popular items
6. Split into Train & Test Data

In [ ]:
# Reading Data Again since it was changed above
df = pd.read_csv('../00_Data/online_retailed_precleaned.csv', index_col=0)

## 1. Flag Column for purchased items:

In [5]:
# Create column 'purchased' and assign the value 1 for each row
df['purchased'] = 1 

# Check First 5 Rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchased
0,536365,3249,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,4048,United Kingdom,1
1,536365,2649,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,1
2,536365,2855,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,4048,United Kingdom,1
3,536365,2803,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,1
4,536365,2802,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,4048,United Kingdom,1


## 2. Decrease value of cancelled purchases 

In [6]:
# First find out Cancelled orders 
# Rows that start with a C in InvoiceNo 
df_cancellations = df[df.InvoiceNo.str[0]=='C']

# Check Head for Sanity
df_cancellations.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchased
154,C536383,2407,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,2192,United Kingdom,1
235,C536391,1415,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,3829,United Kingdom,1
236,C536391,919,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,3829,United Kingdom,1
237,C536391,918,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,3829,United Kingdom,1
238,C536391,915,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,3829,United Kingdom,1
239,C536391,604,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,3829,United Kingdom,1
240,C536391,1416,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,3829,United Kingdom,1
241,C536391,1412,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,3829,United Kingdom,1
939,C536506,1805,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,4088,United Kingdom,1
1441,C536543,1489,HAND WARMER RED RETROSPOT,-1,2010-12-01 14:30:00,2.10,4042,United Kingdom,1


In [11]:
# Filter out unique combination of StockCode & Customer for cancelled Orders 
# Create DataFrmmae
df_cancelled = df_cancellations.groupby(['StockCode', 'CustomerID']).agg({'CustomerID': 'count'})

# Rename the columns
df_cancelled.columns = ['purchased']

# Reset the index
df_cancelled.reset_index(inplace=True)

# Change all cancelled Purchases to -2 
df_cancelled['purchased'] = -2

# Sanity Check with Describe
df_cancelled.describe()

,StockCode,CustomerID,purchased
count,7316.000000,7316.000000,7316.0
mean,1636.699699,1960.620421,-2.0
std,828.131722,1254.610587,0.0
min,7.000000,5.000000,-2.0
25%,1072.000000,848.250000,-2.0
50%,1582.000000,1852.500000,-2.0
75%,2070.000000,2993.000000,-2.0
max,3683.000000,4369.000000,-2.0


In [12]:
# Sample of df_cancelled
df_cancelled.sample(5)

,StockCode,CustomerID,purchased
5068,1981,4213,-2
4541,1809,1523,-2
4563,1813,2821,-2
5419,2041,1728,-2
40,40,3365,-2


## 3. Filter Out cancelled Combinations from Initial Dataset & Concat Cancelled orders to new Dataset

In [30]:
# Perform the anti join by merging on 'StockCode' and 'CustomerID' using a 'left' join and adding an '_merge' indicator column
merged_df = df.merge(df_cancelled, on=['StockCode', 'CustomerID'], how='left', indicator=True)

# Keep only the rows where the '_merge' indicator is 'left_only', meaning they only exist in the 'df' dataframe
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the '_merge' column and any additional columns from 'df_cancelled' that might have been added during the merge
filtered_df = filtered_df.drop(columns=['_merge'])

# Reset the index if needed
filtered_df.reset_index(drop=True, inplace=True)


In [33]:
# Group by InvoiceNo and calculate the number of appearances and the product of Quantity and UnitPrice
df_purchased = filtered_df.groupby(['StockCode', 'CustomerID']).agg({'CustomerID': 'count'})

# Rename the columns
df_purchased.columns = ['purchased']

# Reset the index
df_purchased.reset_index(inplace=True)

# Sanity Check with Describe
df_purchased.head()

,StockCode,CustomerID,purchased
0,0,86,1
1,0,132,1
2,0,191,1
3,0,238,1
4,0,270,1


In [37]:
# Concatenate Purched & Cancelled
df_ratings = pd.concat([df_purchased, df_cancelled], ignore_index=True)

# Check Shape
print(df_ratings.shape)

# First five Rews
df_ratings.head()

(260074, 3)


,StockCode,CustomerID,purchased
0,0,86,1
1,0,132,1
2,0,191,1
3,0,238,1
4,0,270,1


In [38]:
# Describe df_ratings
df_ratings.describe()

,StockCode,CustomerID,purchased
count,260074.000000,260074.000000,260074.000000
mean,1566.765894,2168.561452,1.373209
std,817.726893,1252.728662,1.385767
min,0.000000,1.000000,-2.000000
25%,957.000000,1134.000000,1.000000
50%,1543.000000,2125.000000,1.000000
75%,2102.000000,3272.000000,1.000000
max,3683.000000,4371.000000,54.000000


## 4. Increase value of repurchased items

In [39]:
# Apply function with lambda for the condition: If x in purchased <=1 x, else 2 
df_ratings['purchased'] = df_ratings['purchased'].apply(lambda x: x if x <= 1 else 2)

# Check
df_ratings.purchased.value_counts()



 1    191749
 2     61009
-2      7316
Name: purchased, dtype: int64

## 5. Insert Negative Values for non Purchased Popular Items

In [44]:
# First remove cancellations from initial
df_positive = df[~df['InvoiceNo'].str.startswith('C')]

# Second Count number of Purchases per Item
df_items = df_positive.groupby('StockCode').agg({'StockCode': 'count'})

# Rename the columns
df_items.columns = ['Purchases']

# Reset the index
df_items.reset_index(inplace=True)

df_items.head()


,StockCode,Purchases
0,0,49
1,1,21
2,2,30
3,6,64
4,7,123


In [48]:
# Filter Out best Items 
df_top5_items = df_items[df_items['Purchases'] >= df_items['Purchases'].quantile(0.95)]

# Number of top Items 
df_top5_items.shape[0]

138

In [61]:
# Save top Items in an array 
top5_perc_items = df_top5_items['StockCode'].unique()

# Save all Users from df_rating in an array
all_users = df_ratings['CustomerID'].unique()

In [64]:
# For sanity check in the next step multiply length of both arrays. This should later be the shape of dataframe with unique combinations out of both arrays
len(top5_perc_items) * len(all_users)

510462

In [65]:
# Create Dataframe with all unique matches of top items and all users 

# First Create a meshgrid of all_users and top5_perc_items
user_grid, item_grid = np.meshgrid(all_users, top5_perc_items, indexing='ij')

# Flatten the meshgrid arrays
user_flat = user_grid.flatten()
item_flat = item_grid.flatten()

# Create a pandas dataframe with a row for each combination of the values in all_users and top5_perc_items
top_potential_purchases = pd.DataFrame({'CustomerID': user_flat, 'StockCode': item_flat})

# Check shape: it should be 510462 
top_potential_purchases.shape


(510462, 2)

In [68]:
# Insert a -1 in a new column as value for all rows
top_potential_purchases['purchased'] = -1 

# Sanity check
top_potential_purchases.head()

,CustomerID,StockCode,purchased
0,86,146,-1
1,86,162,-1
2,86,167,-1
3,86,168,-1
4,86,169,-1


In [74]:
# Perform an Anti-Join between df_ratings and top_potential_purchases to find out combinations of CustomerID & StockCode that are only in top_potential purchases. Drop duplicates and concatenate to ratings

# Perform the anti join by merging on 'StockCode' and 'CustomerID' using a 'left' join and adding an '_merge' indicator column
popular_non_bought = top_potential_purchases.merge(df_ratings, on=['StockCode', 'CustomerID'], how='left', indicator=True)

# Keep only the rows where the '_merge' indicator is 'left_only', meaning they only exist in the 'df' dataframe
popular_non_bought = popular_non_bought[popular_non_bought['_merge'] == 'left_only']

# Drop the '_merge' column and any additional columns from 'df_cancelled' that might have been added during the merge
popular_non_bought = popular_non_bought.drop(columns=['_merge', 'purchased_y'])

# Reset the index if needed
popular_non_bought.reset_index(drop=True, inplace=True)

# Rename purchased_x to purchased
popular_non_bought = popular_non_bought.rename(columns={'purchased_x':'purchased'})

# Show first Five rows
popular_non_bought.head()


,CustomerID,StockCode,purchased
0,86,168,-1
1,86,280,-1
2,86,315,-1
3,86,393,-1
4,86,399,-1


In [75]:
# Concatenat epopular_non_bought with df_ratings
df_ratings = pd.concat([df_ratings, popular_non_bought], ignore_index=True)

# Check sample
df_ratings.sample(10)

,StockCode,CustomerID,purchased
149696,1755,1617,1
202859,2186,2914,1
331497,342,3481,-1
363261,1260,898,-1
76194,1069,2368,2
252267,3625,1997,1
652228,2033,2206,-1
93717,1277,815,1
660631,262,3971,-1
400958,891,2207,-1


In [76]:
# Check Descriptive Statistics
df_ratings.describe()

,StockCode,CustomerID,purchased
count,718398.000000,718398.000000,718398.000000
mean,1570.656969,2174.373559,-0.221589
std,845.517794,1261.142384,1.111855
min,0.000000,1.000000,-2.000000
25%,968.000000,1095.000000,-1.000000
50%,1516.000000,2166.000000,-1.000000
75%,2088.000000,3274.000000,1.000000
max,3683.000000,4371.000000,2.000000


In [77]:
# Check Distribution 
df_ratings.purchased.value_counts()

-1    458324
 1    191749
 2     61009
-2      7316
Name: purchased, dtype: int64

## 6. Split Into Training & Test Data

In [79]:
# Load the Chronological Splitter
from recommenders.datasets.python_splitters import python_stratified_split

# Split with Python Stratified
train_rat, test_rat = python_stratified_split(df_ratings, ratio=0.8, filter_by='user', min_rating=10, col_user='CustomerID', col_item='StockCode', seed=1)

In [80]:
# Rows in Train & Test
print("Rows in train:", train_rat.shape[0])
print("Rows in test:", test_rat.shape[0])

Rows in train: 574703
Rows in test: 143695


In [81]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
train_items_rat = set(train_rat['StockCode'].unique())
test_items_rat = set(test_rat['StockCode'].unique())

# Find the intersection of the sets from step 1
common_items_rat = train_items_rat.intersection(test_items_rat)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
train_rat = train_rat[train_rat['StockCode'].isin(common_items_rat)]
test_rat = test_rat[test_rat['StockCode'].isin(common_items_rat)]

In [84]:
#  Check number of unique Users & Items in Test & Train 
print("Users in Test:",test_rat['CustomerID'].nunique())
print("Users in Train:",train_rat['CustomerID'].nunique())
print("Items in Test:",test_rat['StockCode'].nunique())
print("Items in Train:",train_rat['StockCode'].nunique())


Users in Test: 3699
Users in Train: 3699
Items in Test: 2723
Items in Train: 2723


In [85]:
# Rows in Train & Test
print("Rows in train:", train_rat.shape[0])
print("Rows in test:", test_rat.shape[0])

Rows in train: 574361
Rows in test: 143695


In [86]:
# Save as csv 
train_rat.to_csv('../00_Data/online_retail_ratings_train.csv')
test_rat.to_csv('../00_Data/online_retail_ratings_test.csv')